# Drone Routing Optimization Problem

## Model Initialization

In [1]:
# import Glop linear solver package
from ortools.linear_solver import pywraplp as glp
import csv
import numpy as np
import pandas as pd
import time
from datetime import date

# initialize model object
mymodel = glp.Solver('Drone Routing Problem', glp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

## Parameters

### Import distance matrix

In [2]:
# import csv distance matrix as list
with open('Orders_Data.csv', newline='') as f:
    reader = csv.reader(f)
    distance = list(reader)

# remove blanks
for i in range(len(distance)):
    distance[i] = [item for item in distance[i] if item]
distance = [item for item in distance if item]
distance = np.array(distance, dtype = 'float')

# Cost per unit of distance
truck_cost_per_dist = 10.0
drone_cost_per_dist = 3.0

# Create cost matrices for truck & drone
truck_cost = distance * truck_cost_per_dist
drone_cost = distance * drone_cost_per_dist
truck_cost = truck_cost.tolist()
drone_cost = drone_cost.tolist()

In [3]:
distance.tolist()

[[0.0, 100.0, 81.0, 99.0, 61.0, 111.0, 0.0],
 [100.0, 0.0, 97.0, 98.0, 41.0, 93.0, 100.0],
 [81.0, 97.0, 0.0, 19.0, 66.0, 36.0, 81.0],
 [99.0, 98.0, 19.0, 0.0, 74.0, 18.0, 99.0],
 [61.0, 41.0, 66.0, 74.0, 0.0, 77.0, 61.0],
 [111.0, 93.0, 36.0, 18.0, 77.0, 0.0, 111.0],
 [0.0, 100.0, 81.0, 99.0, 61.0, 111.0, 0.0]]

### Initialize Model Parameters

In [4]:
# model parameters
R = list(range(1, len(distance)-1)) # Nodes excluding origin & final 'dummy' node (1 to 10 index)
N = list(range(len(distance))) # All nodes (0 to 11 index)
M = len(distance) + 100 # Large number
D = list(range(3)) # Drones onboard the truck (0 to 2 index)

In [5]:
R

[1, 2, 3, 4, 5]

In [6]:
N

[0, 1, 2, 3, 4, 5, 6]

## Decision Variables

In [7]:
# Truck path from node i to j
dimensions = (int(len(N)), int(len(N))) # Create truck arc dimensions - 11 x 11 (10 x 10 index)
truck_arc = np.zeros(dimensions).tolist() # create nested list
for i in N:
    for j in N:
        truck_arc[i][j] = mymodel.IntVar(0, 1, str(i) + "." + str(j)) #create binary variable for each possible truck path
truck_arc

[[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6],
 [1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6],
 [2.0, 2.1, 2.2, 2.3, 2.4, 2.5, 2.6],
 [3.0, 3.1, 3.2, 3.3, 3.4, 3.5, 3.6],
 [4.0, 4.1, 4.2, 4.3, 4.4, 4.5, 4.6],
 [5.0, 5.1, 5.2, 5.3, 5.4, 5.5, 5.6],
 [6.0, 6.1, 6.2, 6.3, 6.4, 6.5, 6.6]]

In [8]:
# The kth drone path from node i to j
dimensions = (int(len(N)), int(len(N)), int(len(D))) # Create drone arc dimensions - 12 x 12 x 3 (index 11 x 11 x 2)
drone_arc = np.zeros(dimensions).tolist() # create nested list

for i in R:
    for j in R:
        for k in D:
            drone_arc[i][j][k] = mymodel.IntVar(0, 1, str(i) + "." + str(j) + "." + str(k)) #create binary variable for each possible drone path

# All drone paths from node 2 (index 1)
drone_arc[1]

[[0.0, 0.0, 0.0],
 [1.1.0, 1.1.1, 1.1.2],
 [1.2.0, 1.2.1, 1.2.2],
 [1.3.0, 1.3.1, 1.3.2],
 [1.4.0, 1.4.1, 1.4.2],
 [1.5.0, 1.5.1, 1.5.2],
 [0.0, 0.0, 0.0]]

In [9]:
# create position variable (lambda / u)
pos = list(N)
for i in N:
    pos[i] = mymodel.IntVar(0, len(N), "p." + str(i))
pos

[p.0, p.1, p.2, p.3, p.4, p.5, p.6]

In [10]:
# Launch point variable - 1 if the truck goes to node j, 0 otherwise (allows drones to be launched or not from node j)
w = list(N)
for i in R:
    w[i] = mymodel.IntVar(0, 1, "w." + str(i))
w

[0, w.1, w.2, w.3, w.4, w.5, 6]

## Objective

In [11]:
# create objective function
optimal_route = mymodel.Objective()
optimal_route.SetMinimization() # Minimize cost

for i in N:
    for j in N:
        optimal_route.SetCoefficient(truck_arc[i][j], truck_cost[i][j]) # Truck cost per arc
        
for i in R:
    for j in R:
        for k in D:
            optimal_route.SetCoefficient(drone_arc[i][j][k], drone_cost[i][j]) # Drone cost per arc

## Constraints

### Variable Constraints

In [12]:
# Truck variable constraints
#truck_var_1 = list(N)
#for i in N:
#    truck_var_1[i] = mymodel.Constraint(0, 0) # = to 0
#    truck_var_1[i].SetCoefficient(truck_arc[i][i], 1) # Truck cannot go from i to i

In [13]:
#truck_var_2 = list(N)
#for i in N:
#    truck_var_2[i] = mymodel.Constraint(0, 0) # = to 0
#    truck_var_2[i].SetCoefficient(truck_arc[i][0], 1) # Truck cannot go from node i to first node (origin) TAKE OUT

In [14]:
truck_var_3 = list(N)
for i in N:
    truck_var_3[i] = mymodel.Constraint(0, 0) # = to 0
    truck_var_3[i].SetCoefficient(truck_arc[len(N)-1][i], 1) # Truck cannot go from last node to i

In [15]:
#truck_var_4 = mymodel.Constraint(0,0)
#truck_var_4.SetCoefficient(truck_arc[0][len(N)-1], 1)

In [16]:
# Drone variable constraints
#dimensions_2 = (int(len(N)), int(len(D))) # Create drone var dimensions - 12 x 3 (index 11 x 2)
#drone_var = np.zeros(dimensions_2).tolist() # create nested list
#for i in R:
#    for k in D:
#        drone_var[i][k] = mymodel.Constraint(0, 0) # = to 0
#        drone_var[i][k].SetCoefficient(drone_arc[i][i][k], 1) # Drone cannot go from i to i

### Truck arc constraints

In [17]:
# First arc - Don't think I need
first_arc = mymodel.Constraint(1, 1) # = to 1
for i in N:
    first_arc.SetCoefficient(truck_arc[0][i], 1) # Truck must leave the origin and go somewhere (might be able to take out a truck var constraint or 2 if we change this to R instead of N)

In [18]:
# Last arc - Definitely don't need
last_arc = mymodel.Constraint(1, 1) # = to 1
for i in N:
    last_arc.SetCoefficient(truck_arc[i][len(N)-1], 1) # Truck must come from somewhere and return to the origin

In [19]:
# Truck balance - In = Out
truck_bal = list(N)
for i in R:
    truck_bal[i] = mymodel.Constraint(0, 0)
    for j in R:
        truck_bal[i].SetCoefficient(truck_arc[i][j], 1)
        truck_bal[i].SetCoefficient(truck_arc[j][i], -1)
        
dimensions_3 = (int(len(N)), int(len(N))) # Create MTZ constraint dimensions - 12 x 12 (index 11 x 11)
MTZ = np.zeros(dimensions_3).tolist() # create nested list

for i in N:
    for j in N:
        MTZ[i][j] = mymodel.Constraint(-mymodel.infinity(), M-1)
        MTZ[i][j].SetCoefficient(truck_arc[i][j], M)
        MTZ[i][j].SetCoefficient(pos[i], 1)
        MTZ[i][j].SetCoefficient(pos[j], -1)

In [20]:
truck_bal

[0,
 <ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x0000014DBAA5FED0> >,
 <ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x0000014DBAA5FE70> >,
 <ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x0000014DBAA5FF00> >,
 <ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x0000014DBAA5FF60> >,
 <ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x0000014DBAA5FFC0> >,
 6]

### Truck & Drone Covering Constraint

In [21]:
# All intermediate nodes must be visited by either a truck or a drone
covering = list(N)
for j in R:
    covering[j] = mymodel.Constraint(1, mymodel.infinity()) # constraint for each intermediate node j in R (1 to 10 index) - can we make it from 1 to N and delete some others?
    for i in N: # index from 0 to 11
        covering[j].SetCoefficient(truck_arc[i][j], 1) # Truck must go to node j if a drone doesn't
    for i in R: # Different index for i here (1 to 10)
        for k in D:
            covering[j].SetCoefficient(drone_arc[i][j][k], 1) # A drone must go to node j if the truck doesn't

### Drone Constraints

In [22]:
# Drones must leave & come back to the same node
dimensions_4 = (int(len(N)), int(len(N)), int(len(D))) # Create launch point constraint dimensions - 12 x 12 x 3 (index 11 x 11 x 2)
launch = np.zeros(dimensions_4).tolist() # create nested list
for i in R:
    for j in R:
        for k in D:
            launch[i][j][k] = mymodel.Constraint(0, 0)
            launch[i][j][k].SetCoefficient(drone_arc[i][j][k], 1) # Drone k goes from i to j 
            launch[i][j][k].SetCoefficient(drone_arc[j][i][k], -1) # Drone k goes from j back to i

In [23]:
# Recycling drone constraint - at each node i, drone k can only be used to go to one node j. (not multiple j's)
dimensions_5 = (int(len(N)), int(len(D))) # Create node origin constraint dimensions - 12 (index 11)
drone_recy = np.zeros(dimensions_5).tolist() # create nested list
for i in R:
    drone_recy[i] = mymodel.Constraint(-mymodel.infinity(), 1) # for each drone k at each node i
    for j in R:
        for k in D:
            drone_recy[i].SetCoefficient(drone_arc[i][j][k], 1) # sum of all routes out to node j should be <= 1

### Position Constraints

In [24]:
#position 1 = 1 (lambda for MTZ) - not sure if I need this
position_1 = mymodel.Constraint(1, 1)
position_1.SetCoefficient(pos[0], 1)

#position N = N (lambda for MTZ) - Don't use 
#position_N = mymodel.Constraint(1, 1)
#position_N.SetCoefficient(pos[len(N)-1], len(N))

In [25]:
# Launch point constraint - allows drones to be launched if a truck has visited that node
launch_point = list(N)
for j in R:
    launch_point[j] = mymodel.Constraint(0, 0)
    launch_point[j].SetCoefficient(w[j], -1) # connecting variable - 1 if the truck goes to the node, 0 otherwise
    for i in N:
        launch_point[j].SetCoefficient(truck_arc[i][j], 1) # all truck routes leading to that node

In [26]:
# Floating drone constraint - not sure that we need this but it's here
dimensions_6 = (int(len(N)), int(len(N))) # Create node origin constraint dimensions - 12 (index 11)
floating_drone = np.zeros(dimensions_6).tolist() # create nested list
for i in R:
    for j in R:
        floating_drone[i][j] = mymodel.Constraint(-mymodel.infinity(), 0)
        floating_drone[i][j].SetCoefficient(w[i], -1)
        floating_drone[i][j].SetCoefficient(w[j], -1)
        for k in D:
            floating_drone[i][j].SetCoefficient(drone_arc[i][j][k], 1)

In [27]:
# Solve the model and print optimal solution
start_time = time.time()
status = mymodel.Solve()                 # solve mymodel and display the solution

print('Solution Status =', status)
print('Number of variables =', mymodel.NumVariables())
print('Number of constraints =', mymodel.NumConstraints())

print('Optimal Solution:')

# The objective value of the solution.
print('Optimal Value = %.2f' % optimal_route.Value())

print("Solution Time: %.2f seconds" % (time.time() - start_time))
runtime = round(time.time() - start_time)
today = str(date.today())

# Display optimal solution
for i in N:
    print('pos[%d] = %d' % (i, pos[i].solution_value()))
    for j in N:
        if truck_arc[i][j].solution_value() == 1:
            print('Truck ', i, ' to ', j, ': ', truck_arc[i][j].solution_value(), sep = '')
for i in R:
    for j in R:
        for k in D:
            if drone_arc[i][j][k].solution_value() == 1:
                print('Drone_', k, ' from ', i, ' to ', j, ': ', drone_arc[i][j][k].solution_value(), sep = '')

for j in R:
    print('Drone launch point ', j, ': ', w[j].solution_value(), sep = '')


    
#        if i == N-1 or i == j:
#            continue
#        elif j == N-1:
 #           j = 1
 #           print('From node ', i + 1, ' to node ', j + 1, ': ', use_arc[j][i].solution_value(), sep = '')
 #       else:
  #          print('From node ', i + 1, ' to node ', j + 1, ': ', use_arc[j][i].solution_value(), sep = '') 

Solution Status = 2
Number of variables = 136
Number of constraints = 179
Optimal Solution:
Optimal Value = 0.00
Solution Time: 1.47 seconds
pos[0] = 0
pos[1] = 0
pos[2] = 0
pos[3] = 0
pos[4] = 0
pos[5] = 0
pos[6] = 0
Drone launch point 1: 0.0
Drone launch point 2: 0.0
Drone launch point 3: 0.0
Drone launch point 4: 0.0
Drone launch point 5: 0.0


In [28]:
From_truck = []
To_truck = []


for i in N:
    for j in N:
        if truck_arc[i][j].solution_value() == 1:
            From_truck.append(i)
            To_truck.append(j)


                
truck_solution = pd.DataFrame(list(zip(From_truck, To_truck)), columns = ['From', 'To'])
#solution = solution.sort_values('Position').reset_index(drop=True)
#solution
truck_solution

,From,To


In [29]:
From_drone = []
To_drone = []
Drone_num = []

for i in R:
    for j in R:
        for k in D:
            if drone_arc[i][j][k].solution_value() == 1:
                From_drone.append(i)
                To_drone.append(j)
                Drone_num.append(k)
drone_solution = pd.DataFrame(list(zip(From_drone, To_drone, Drone_num)), columns = ['From', 'To', 'Drone'])
#solution = solution.sort_values('Position').reset_index(drop=True)
#solution
drone_solution

,From,To,Drone
